In [1]:
using JuMP, Gurobi

m = Model(Gurobi.Optimizer)
set_optimizer_attribute(m, "TimeLimit", 300)

set_optimizer_attribute(m, "OutputFlag", 1)
set_optimizer_attribute(m, "LogFile", "gurobi_log.txt")

Set parameter Username
Academic license - for non-commercial use only - expires 2025-08-21
Set parameter TimeLimit to value 300
Set parameter LogFile to value "gurobi_log.txt"


In [2]:
# number of weeks and days
T = 8
N = 7

# historical heart rate and weights for objective terms
HR_hist = 150
w1, w2, w3, w4 = 0.3, 1, 0.7, 0.37

# race distance and minimum total training distance
Race_Distance = 6.2
Min_Total_Distance = 31.06

# weekly distance progression limits
MaxWeeklyDist_hist = 10
Delta_Weekly_Distance = 1.5

# pacing history and weekly pace progression
FastestPace_hist = 10.0
Delta_Weekly_Pace = 0.02

# max distance per run and min distance per run when xt=1
Md, ed = 6.2, 0.5

# max and min pace per run when xt=1
Mp, ep = 20.0, 4.0

# define long run progression parameters
LongRun_InitialDistance = Race_Distance * 0.5       # half race distance start
LongRun_FinalDistance = 0.85 * Race_Distance        # target ~85% race distance for long run
Delta_longrun = (LongRun_FinalDistance - LongRun_InitialDistance) / (T - 1) 
# incremental increase in long run distance each week

0.30999999999999994

In [3]:
# binary variables for training days
@variable(m, xt[1:T, 1:N], Bin)

# continuous daily distance (miles); not integers for flexibility
@variable(m, dt[1:T, 1:N] >= 0)

# continuous pace (min/mile)
@variable(m, pt[1:T, 1:N] >= 0)

# auxiliary variable for pace-readiness linking (unused in final constraints but kept)
@variable(m, st[1:T, 1:N] >= 0)

# binary variables reserved for other logic (yt for shortest/fastest runs, currently unused)
@variable(m, yt[1:T, 1:N], Bin)

# binary variables to identify specific long run conditions (zt)
@variable(m, zt[1:T], Bin)

# continuous variable for z_pace to track pace-readiness metric
@variable(m, z_pace >= 0)

# numerator and denominator for ratio calculations related to z_pace
@variable(m, numerator >= 0)
@variable(m, denominator >= 0)

# binary variables to identify tempo runs each week (yr), only for days 2 to N
@variable(m, yr[1:T, 2:N], Bin)

2-dimensional DenseAxisArray{VariableRef,2,...} with index sets:
    Dimension 1, Base.OneTo(8)
    Dimension 2, 2:7
And data, a 8×6 Matrix{VariableRef}:
 yr[1,2]  yr[1,3]  yr[1,4]  yr[1,5]  yr[1,6]  yr[1,7]
 yr[2,2]  yr[2,3]  yr[2,4]  yr[2,5]  yr[2,6]  yr[2,7]
 yr[3,2]  yr[3,3]  yr[3,4]  yr[3,5]  yr[3,6]  yr[3,7]
 yr[4,2]  yr[4,3]  yr[4,4]  yr[4,5]  yr[4,6]  yr[4,7]
 yr[5,2]  yr[5,3]  yr[5,4]  yr[5,5]  yr[5,6]  yr[5,7]
 yr[6,2]  yr[6,3]  yr[6,4]  yr[6,5]  yr[6,6]  yr[6,7]
 yr[7,2]  yr[7,3]  yr[7,4]  yr[7,5]  yr[7,6]  yr[7,7]
 yr[8,2]  yr[8,3]  yr[8,4]  yr[8,5]  yr[8,6]  yr[8,7]

In [4]:
# define the objective to maximize a weighted sum of total distance, pace-readiness, rest days, and training frequency
@objective(m, Max,
    w1 * sum(dt[t, n] for t in 1:T, n in 1:N) +
    w2 * HR_hist * z_pace +
    w3 * sum(7 - sum(xt[t, n] for n in 1:N) for t in 1:T) +
    w4 * sum(xt[t, n] for t in 1:T, n in 1:N)
)

0.3 dt[1,1] + 0.3 dt[1,2] + 0.3 dt[1,3] + 0.3 dt[1,4] + 0.3 dt[1,5] + 0.3 dt[1,6] + 0.3 dt[1,7] + 0.3 dt[2,1] + 0.3 dt[2,2] + 0.3 dt[2,3] + 0.3 dt[2,4] + 0.3 dt[2,5] + 0.3 dt[2,6] + 0.3 dt[2,7] + 0.3 dt[3,1] + 0.3 dt[3,2] + 0.3 dt[3,3] + 0.3 dt[3,4] + 0.3 dt[3,5] + 0.3 dt[3,6] + 0.3 dt[3,7] + 0.3 dt[4,1] + 0.3 dt[4,2] + 0.3 dt[4,3] + 0.3 dt[4,4] + 0.3 dt[4,5] + 0.3 dt[4,6] + 0.3 dt[4,7] + 0.3 dt[5,1] + 0.3 dt[5,2] + [[...53 terms omitted...]] - 0.32999999999999996 xt[4,6] - 0.32999999999999996 xt[4,7] - 0.32999999999999996 xt[5,1] - 0.32999999999999996 xt[5,2] - 0.32999999999999996 xt[5,3] - 0.32999999999999996 xt[5,4] - 0.32999999999999996 xt[5,5] - 0.32999999999999996 xt[5,6] - 0.32999999999999996 xt[5,7] - 0.32999999999999996 xt[6,1] - 0.32999999999999996 xt[6,2] - 0.32999999999999996 xt[6,3] - 0.32999999999999996 xt[6,4] - 0.32999999999999996 xt[6,5] - 0.32999999999999996 xt[6,6] - 0.32999999999999996 xt[6,7] - 0.32999999999999996 xt[7,1] - 0.32999999999999996 xt[7,2] - 0.329999999

In [5]:
# set minimum total distance over the entire training period
@constraint(m, sum(dt[t, n] for t in 1:T, n in 1:N) >= Min_Total_Distance)

# set maximum weekly distance progression
@constraint(m, [t=1:T], sum(dt[t, n] for n in 1:N) <= MaxWeeklyDist_hist + (t - 1) * Delta_Weekly_Distance)

# set taper week: final week's mileage ≤ 80% of the previous week
@constraint(m, sum(dt[T, n] for n in 1:N) <= 0.8 * sum(dt[T-1, n] for n in 1:N))

# ensure exactly one tempo run (yr=1) per week (excluding day 1)
@constraint(m, [t=1:T], sum(yr[t,n] for n=2:N) == 1)

# long run pace ≤ 1.2x historic fastest adjusted per week
@constraint(m, [t=1:T], pt[t,1] ≤ 1.2 * (FastestPace_hist - (t-1)*Delta_Weekly_Pace) * xt[t,1])

# tempo run pace ≤ fastest adjusted pace if yr=1; otherwise ≤ 1.1x adjusted pace
@constraint(m, [t=1:T, n=2:N],
    pt[t,n] ≤ (FastestPace_hist - (t-1)*Delta_Weekly_Pace)*yr[t,n] + 
              1.1*(FastestPace_hist - (t-1)*Delta_Weekly_Pace)*(1 - yr[t,n]) 
)

# tempo run day must be a training day
@constraint(m, [t=1:T, n=2:N], yr[t,n] ≤ xt[t,n])

# ≤5 training days per week
@constraint(m, [t=1:T], sum(xt[t, n] for n in 1:N) <= 5)

# no run on day 2 if day 1 is a run
@constraint(m, [t=1:T], xt[t,1] + xt[t,2] <= 1)

# long run ≥ each other run in that week; non-long runs ≤3.1 miles
@constraint(m, [t=1:T, n=2:N], dt[t, 1] >= dt[t, n])
@constraint(m, [t=1:T, n=2:N], dt[t, n] <= 3.1)

# long run progression upper bound (no strict lower bound)
@constraint(m, [t=1:T], dt[t, 1] <= LongRun_InitialDistance + (t-1)*Delta_longrun + 0.5)

# ensure at least one long run hits 80%-100% range (zt indicates weeks with qualifying long run)
@constraint(m, sum(zt[t] for t in 1:T) >= 1)
@constraint(m, [t=1:T], dt[t, 1] >= ed * zt[t])
@constraint(m, [t=1:T], dt[t, 1] <= LongRun_FinalDistance * zt[t])

# taper the final week's long run to ≤80% of previous week's long run
@constraint(m, dt[T, 1] <= 0.8 * dt[T-1, 1])

# link distance and runs: distance only if xt=1; pace only if xt=1
@constraint(m, [t=1:T, n=1:N], dt[t, n] <= Md * xt[t, n])
@constraint(m, [t=1:T, n=1:N], dt[t, n] >= ed * xt[t, n])
@constraint(m, [t=1:T, n=1:N], pt[t, n] <= Mp * xt[t, n])
@constraint(m, [t=1:T, n=1:N], pt[t, n] >= ep * xt[t, n])

# pace-readiness linking: numerator/denominator for z_pace, encouraging better pace
@constraint(m, numerator == sum(xt[t, n] * pt[t, n] for t in 1:T, n in 1:N))
@constraint(m, denominator == sum(xt[t, n] for t in 1:T, n in 1:N))
@constraint(m, numerator >= z_pace * denominator)

-z_pace*denominator + numerator ≥ 0

In [6]:
optimize!(m)

status = termination_status(m)
if status == MOI.OPTIMAL
    println("Optimal solution found!")
elseif status == MOI.TIME_LIMIT
    println("Solver reached the time limit.")
else
    println("Solver did not converge: ", status)
end

println("\n10K Training Plan:")
if status == MOI.OPTIMAL
    for t in 1:T
        if t == T
            println("Week $t (Taper Week):")
        else
            println("Week $t:")
        end

        tempo_day = 0
        for n in 2:N
            if value(yr[t,n]) > 0.5
                tempo_day = n
                break
            end
        end

        for n in 1:N
            run_status = value(xt[t, n])
            if run_status > 0.5
                distance = value(dt[t, n])
                raw_pace = value(pt[t, n])

                pace_minutes = Int(floor(raw_pace))
                pace_seconds = Int(round((raw_pace - floor(raw_pace))*60))
                sec_str = lpad(string(pace_seconds), 2, '0')

                labels = ""
                if n == 1
                    labels *= "(long run)"
                elseif n == tempo_day
                    labels *= "(tempo)"
                end

                println("  Day $n: $(round(distance, digits=1)) miles, $pace_minutes:$sec_str pace $labels")
            else
                println("  Day $n: Rest day")
            end
        end
        println("")
    end
else
    println("No solution available.")
end

Set parameter TimeLimit to value 300
Set parameter LogFile to value "gurobi_log.txt"
Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (mac64[arm] - Darwin 23.4.0 23E214)

CPU model: Apple M3 Pro
Thread count: 12 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 485 rows, 339 columns and 1153 nonzeros
Model fingerprint: 0x28f24c5a
Model has 2 quadratic constraints
Variable types: 171 continuous, 168 integer (168 binary)
Coefficient statistics:
  Matrix range     [5e-01, 2e+01]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [3e-01, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 3e+01]
Presolve removed 229 rows and 161 columns
Presolve time: 0.00s
Presolved: 379 rows, 218 columns, 1090 nonzeros
Presolved model has 1 bilinear constraint(s)

Solving non-convex MIQCP

Variable types: 137 continuous, 81 integer (80 binary)

Root relaxation: objective 5.459091e+03, 245 iterations, 0.00 seconds (0.00